In [299]:
import time
import copy
import numpy as np
import pandas as pd
import random
import chainer
import chainer.functions as F
import chainer.links as L
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl

import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [300]:
data = pd.read_csv('gongames_df.csv')
data

,date_hour,impressions
0,2021-01-17,13430
1,2021-01-18,13668
2,2021-01-19,11065
3,2021-01-20,9554
4,2021-01-21,9306
...,...,...
169,2021-07-05,9518
170,2021-07-06,9374
171,2021-07-07,10563
172,2021-07-08,11698


In [301]:
for i in range(len(data)):
    data['date_hour'][i] = data['date_hour'][i].split()[0]
data

<ipython-input-301-a2b8ba5a4b69>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date_hour,impressions
0,2021-01-17,13430
1,2021-01-18,13668
2,2021-01-19,11065
3,2021-01-20,9554
4,2021-01-21,9306
...,...,...
169,2021-07-05,9518
170,2021-07-06,9374
171,2021-07-07,10563
172,2021-07-08,11698


In [302]:
data['date_hour'] = pd.to_datetime(data['date_hour'], format='%Y-%m-%d')
data

,date_hour,impressions
0,2021-01-17,13430
1,2021-01-18,13668
2,2021-01-19,11065
3,2021-01-20,9554
4,2021-01-21,9306
...,...,...
169,2021-07-05,9518
170,2021-07-06,9374
171,2021-07-07,10563
172,2021-07-08,11698


In [303]:
data = pd.DataFrame(data.groupby('date_hour')['impressions'].agg('sum'), columns=['a', 'impressions'])
data.drop(['a'], axis=1, inplace=True)

In [304]:
data

,impressions
date_hour,
2021-01-17,13430
2021-01-18,13668
2021-01-19,11065
2021-01-20,9554
2021-01-21,9306
...,...
2021-07-05,9518
2021-07-06,9374
2021-07-07,10563


In [305]:
data2 = pd.read_csv('gongames_df2.csv')
data2.set_index('date_hour', inplace=True, drop=True)
data2

,impressions
date_hour,
2021-01-17,6669
2021-01-18,181
2021-01-19,5839
2021-01-20,14736
2021-01-21,26672
...,...
2021-07-05,8409
2021-07-06,7712
2021-07-07,11210


In [306]:
learning_rate = 0.0001
gamma = 0.9
buffer_limit = 5000
batch_size = 64
n_rollout = 10

In [332]:
#환경 구축
class Environment:
    
    def __init__(self, data1, data2, budget, history_t=174):
        self.data1 = data1 #twee 1
        self.data2 = data2 #twee 2
        self.budget = budget #광고 집행 예산
        self.history_t = history_t #에피소드별 state 수
        self.reset() #초기화
        
    def reset(self):
        self.t = 0 #현재 state
        self.done = False
        self.positions_a = []
        self.positions_b = []
        self.positions_sum = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history
    
    def step(self, act):
        reward = 0
        ad_a = 0
        ad_b = 0
        self.positions_sum.append(self.data1['impressions'][self.t] + self.data2['impressions'][self.t])
        self.positions_a.append(self.data1['impressions'][self.t])
        self.positions_b.append(self.data2['impressions'][self.t])
        rate_a = self.data1['impressions'][self.t]/self.positions_sum[self.t] * self.budget
        rate_b = self.data2['impressions'][self.t]/self.positions_sum[self.t] * self.budget

        if act == 0: # redistribute
            ad_a = random.randrange(1000, self.budget, 5000) 
            ad_b = self.budget - ad_a
            if self.positions_sum[self.t-1] < self.positions_sum[self.t]:
                if ad_a in range(int(rate_a*0.8), int(rate_a*1.2)) and ad_b in range(int(rate_b*0.8), int(rate_b*1.2)):
                    reward += 1
                else:
                    reward -= 1
        elif act == 1: #a+b= 1000
            ad_a = random.randrange(0, self.budget, 1000) #예산 5000원 (a impressions - 100) - 100000 광고 예산으로 쓸때 1만원 이하의 광고 집행비용을 사용하는게 효율적
            ad_b = self.budget - ad_a #95000원
            if ad_a in range(0, int(rate_a)) and self.positions_a[self.t] in range(0, int((float(self.positions_sum[self.t])*0.1))):
                reward += 1
            else:
                reward -= 1
            if ad_b in range(0, int(rate_a)) and self.positions_b[self.t] in range(0, int((float(self.positions_sum[self.t])*0.1))):
                reward += 1
            else:
                reward -= 1
            
        # set next time
        # self.t += 1
        # self.position_value = 0
        for _ in self.positions_sum:
            self.position_value += (self.data1['impressions'][self.t] + self.data2['impressions'][self.t])
            self.t += 1
        self.history.pop(0)
        self.history.append((self.data1['impressions'][self.t] + data2['impressions'][self.t]) - (self.data1['impressions'][self.t-1] + data2['impressions'][self.t-1]))
        # self.history.append(self.data1.iloc[self.t, :]['impressions'] - self.data1.iloc[(self.t-1), :]['impressions'])
        
        # clipping reward
        if reward > 0:   
            reward += 1
        elif reward < 0:
            reward -= 1
        return '보상 :', reward, '광고 A :', ad_a, '광고 B :', ad_b # obs, reward

In [333]:
env = Environment(data, data2, 100000)
print(Environment(data, data2, 100000).reset())
for _ in range(10):
    pact = np.random.randint(2)
    print('예산 분배 - 10만원')
    print(Environment(data, data2, 100000).step(pact))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
예산 분배 - 10만원
('보상 :', -3, '광고 A :', 47000, '광고 B :', 53000)
예산 분배 - 10만원
('보상 :', -3, '광고 A :', 25000, '광고 B :', 75000)
예산 분배 - 10만원
('보상 :', -3, '광고 A :', 46000, '광고 B :', 54000)
예산 분배 - 10만원
('보상 :', 0, '광고 A :', 91000, '광고 B :', 9000)
예산 분배 - 10만원
('보상 :', -3, '광고 A :', 70000, '광고 B :', 30000)
예산 분배 - 10만원
('보상 :', 0, '광고 A :', 11000, '광고 B :', 89000)
예산 분배 - 10만원
('보상 :', -3, '광고 A :', 22000, '광고 B :', 78000)
예산 분배 - 10만원
('보상 :', -3, '광고 A :', 63000, '광고 B :', 3700

In [319]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(1*(len(data)), 256)
        self.fc2 = nn.Linear(256, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
    
    def put_data(self, item):
        self.data.append(item)

    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r * gamma * R
            loss = -R * torch.log(prob)
            # loss.backword()
        self.optimizer.step()
        self.data = []

In [320]:
# def main():
#     env = Environment(data, data2, 10000)
#     pi = Policy()
#     score = 0.0
#     print_interval = 20

#     for n_epi in range(10):
#         s = env.reset()
#         done = False

#         while not done:
#             prob = pi(torch.from_numpy(np.array(s)). float())
#             m = Categorical(prob)
#             a = m.sample()
#             s_prime, r, done, info = env.step(a.item())
#             pi.put_data((r, prob[a]))
#             s = s_prime
#             score += r
        
#         pi.train_net()
#         if n_epi%print_interval==0 and n_epi!=0:
#             print('#of episdoe : {}, avg score : {}'.format(n_epi, score/print_interval))
#             score = 0.0
#     # env.close()

In [321]:
# main()

In [322]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(1*len(data)+1, 256)
        self.fc_pi = nn.Linear(256, 2)
        self.fc_v = nn.Linear(256, 1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self, x, softmax_dim=0):
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim)
        return prob
    
    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v

    def put_data(self, transition):
        self.data.append(transition)

    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, done_lst = [], [], [], [], []
        for transition in self.data:
            s,a,r,s_prime,done = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r/100.0])
            
            s_prime_lst.append(s_prime)
            done_mask = 0.0 if done else 1.0
            done_lst.append([done_mask])

        s_batch, a_batch, r_batch, s_prime_batch, done_batch = torch.tensor(s_lst, dtype=torch. float), torch.tensor(a_lst), torch.tensor(r_lst, dtype=torch.float), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_lst, dtype=torch. float)
        self.data = []
        return s_batch, a_batch, r_batch, s_prime_batch, done_batch

    def train_net(self):
        s, a, r, s_prime, done = self.make_batch()
        td_target = r + gamma * self.v(s_prime) * done
        delta = td_target - self.v(s)

        pi = self.pi(s, softmax_dim=1)
        pi_a = pi.gather(1, a)
        loss = -torch.log(pi_a) * delta.detach() + F.smooth_l1_loss(self.v(s), td_target.detach())

        self.optimizer.zero_grad()
        # loss.mean().backword()
        self.optimizer.step()

In [323]:
def main2():
    env = Environment(data, data2, 1000000)
    model = ActorCritic()
    print_interval = 20
    score = 0.0

    for n_epi in range(10000):
        done = False
        s = env.reset()

        while not done:
            for t in range(n_rollout):
                prob = model.pi(torch.from_numpy(np.array(s)). float())
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                model.put_data((s,a,r,s_prime,done))
                s = s_prime
                score += r
                if done:
                    break
            
            model.train_net()
        if n_epi%print_interval==0 and n_epi!=0:
            print('#of episdoe : {}, avg score : {}'.format(n_epi, score/print_interval))
            score = 0.0
    # # env.close()

In [324]:
main2()

#of episdoe : 20, avg score : -1.8
#of episdoe : 40, avg score : -1.35
#of episdoe : 60, avg score : -1.35
#of episdoe : 80, avg score : -1.95
#of episdoe : 100, avg score : -1.2
#of episdoe : 120, avg score : -0.75
#of episdoe : 140, avg score : -1.5
#of episdoe : 160, avg score : -1.5
#of episdoe : 180, avg score : -1.95
#of episdoe : 200, avg score : -0.9
#of episdoe : 220, avg score : -1.2
#of episdoe : 240, avg score : -1.65
#of episdoe : 260, avg score : -1.5
#of episdoe : 280, avg score : -2.25
#of episdoe : 300, avg score : -1.35
#of episdoe : 320, avg score : -2.4
#of episdoe : 340, avg score : -1.65
#of episdoe : 360, avg score : -1.2
#of episdoe : 380, avg score : -1.65
#of episdoe : 400, avg score : -1.65
#of episdoe : 420, avg score : -1.35
#of episdoe : 440, avg score : -0.9
#of episdoe : 460, avg score : -1.35
#of episdoe : 480, avg score : -1.05
#of episdoe : 500, avg score : -1.5
#of episdoe : 520, avg score : -1.65
#of episdoe : 540, avg score : -1.2
#of episdoe : 560